In [2]:
import matplotlib.pyplot as plt
# 한글 폰트 지정
plt.rc("font", family = "Malgun Gothic")
# 스타일 서식 train
plt.style.use('ggplot')

# 마이너스 기호 표시 설정
plt.rcParams['axes.unicode_minus'] = False

In [4]:
# 필요한 라이브러리 임포트
# 데이터 처리 및 시각화 라이브러리
import pandas as pd
import seaborn as sns
import numpy as np
import random
import os
from matplotlib.pylab import rcParams  # 차트의 크기를 조절하기 위한 라이브러리

# 사이킷런 (scikit-learn) 라이브러리
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV  # 파라미터 설정 고민을 줄여주는 고마운 친구
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error, r2_score  # loss function 커스터마이징
from sklearn.cluster import KMeans
from sklearn.tree import export_graphviz, DecisionTreeClassifier

# 통계 및 시계열 분석 라이브러리
from statsmodels.tsa.seasonal import seasonal_decompose  # 시계열
from statsmodels.tsa.stattools import adfuller  # 정상성 테스트
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf  # PACF, ACF
from statsmodels.tsa.arima.model import ARIMA  # ARIMA 모델

# 머신러닝 라이브러리 (LightGBM, XGBoost)
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import plot_importance, XGBClassifier, plot_tree

# 딥러닝 라이브러리 (PyTorch)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 기타
from pandas.plotting import autocorrelation_plot
from tqdm.auto import tqdm
import graphviz
import warnings
import shap
from pycaret.classification import *
import ssl 
from pdpbox import info_plots

warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

ModuleNotFoundError: No module named 'statsmodels'